In [ ]:
!pip install flask_sqlalchemy

In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install pyngrok==4.1.1

In [ ]:
!ngrok authtoken '294dzGF6SRGFWn5l0brOtz37HvR_2WRzjVis8V5QuYW9A7Gda'

In [ ]:
!pip install ffmpeg-python

In [ ]:
#!pip install DBCreateScript

In [ ]:
from flask import Flask, render_template, request, jsonify, flash
from flask_ngrok import run_with_ngrok
from flask_sqlalchemy import SQLAlchemy
# from DBCreateScript import db_object
from google.colab import drive
import json
from flask import Response
from functools import wraps

drive.mount('/content/gdrive')

import urllib
import os
import sys
import shutil
sys.path.append('/content/gdrive//MyDrive/VoiceRecognition/main/')
import record_voice
import Predict
import newlearn

TEMPLATE_FOLDER = '/content/gdrive//MyDrive/VoiceRecognition/templates' # 임의의 경로
STATIC_FOLDER = '/content/gdrive//MyDrive/VoiceRecognition/static' # 임의의 경로
app = Flask(__name__, template_folder=TEMPLATE_FOLDER, static_folder = STATIC_FOLDER)
run_with_ngrok(app)
app.config['JSON_AS_ASCII'] = False
app.config["SECRET_KEY"] = "ABCD"

# app.config['SQLALCHEMY_DATABASE_URI'] = db_object.connection()
# app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False

#db = SQLAlchemy(app)

def as_json(f):
  @wraps(f)
  def decorated_function(*args, **kwargs):
    res = f(*args, **kwargs)
    res = json.dumps(res, ensure_ascii=False).encode('utf8')
    return Response(res, content_type='application/json; charset=utf-8')
  return decorated_function

#목소리 등록 버튼 클릭 시
@app.route('/register', methods=['POST'])
def register_voice():
  data = request.get_json()
  name = data['name']
  age = data['age']
  sex = data['sex']

  print(name+age+sex)

  record_voice.record(name,age,sex)

  return jsonify(result="success")

#등록된 목소리 확인 버튼 클릭 시
@app.route('/voicecheck/<data>')
def check_voice(data):
  user_info=data
  print(user_info)

  voicelist=[]

  path = "/content/gdrive//MyDrive/VoiceRecognition/static/Recording/"

  if not os.path.isdir(path+user_info):
    flash("등록된 목소리가 없습니다. 목소리 등록을 시도해주십시오.")
    return render_template('Checkvoice.html')
  else:
    for list in os.listdir(path+user_info):
      voicelist.append('/static/Recording/'+user_info+'/'+list)
      print('/Recording/'+user_info+'/'+list)
    print(voicelist)
    return render_template('Checkvoice.html',voicelist=voicelist)

#화자인식 버튼 클릭 시
@app.route('/recognition/<data>')
def regcognition(data):
  user_info=data

  path = "/content/gdrive//MyDrive/VoiceRecognition/static/Recording/"

  if not os.path.isdir(path+user_info):
    flash("등록된 목소리가 없습니다. 목소리 등록을 시도해주십시오.")
    return render_template('Recognition.html')
  else:
    print("화자인식 버튼 클릭 후 화자인식 진행창 띄우기")
    print(user_info)

    return render_template('Recognition.html',user_info=user_info)

#화자인식 페이지의 인식 버튼 클릭 시
@app.route('/recording_recognition', methods=['POST'])
def recording_recognition():
  data = request.get_json() #김민지22female
  user_info=urllib.parse.unquote(data)
  print(user_info)

  recognition_audio=record_voice.recognition_record()

  print("화자인식 녹음 종료")

  #화자인식 메소드
  result=Predict.result_recognition(user_info)

  return jsonify(result=result)

#사용자 삭제 버튼 클릭 시
@app.route('/delete_user/<data>')
def delete_user(data):
  user_info=data
  path = "/content/gdrive//MyDrive/VoiceRecognition/static/Recording/"

  print(user_info)

  if not os.path.isdir(path+user_info):
    print("등록X")
    flash("기존에 등록되지 않은 사용자 입니다.")
    return render_template('Recognition.html')
    # return jsonify(result="false")
  else:
    shutil.rmtree(path+user_info)
    flash("해당 사용자가 삭제되었습니다.")
    return render_template('Recognition.html')

  # return jsonify(result="success")

#Train 버튼 클릭 시
@app.route('/training')
def training():
  model_path = "/content/gdrive//MyDrive/VoiceRecognition/main/demo_model.h5"
  record_path = "/content/gdrive//MyDrive/VoiceRecognition/static/Recording/"
  newlearn.model_update(model_path, record_path)
  return render_template('Registration.html')

@app.route('/')
def home():
  return render_template('Registration.html')

if __name__ == '__main__':
  app.run()